In [7]:
import mysql.connector
from tabulate import tabulate

In [9]:
mycursor = mydb.cursor()
mycursor.execute("USE mavenmovies")

mycursor.execute("SELECT store_id,last_name,first_name FROM customer LIMIT 5 ")
myresult = mycursor.fetchall()


print(tabulate(myresult, headers = ['store_id','last_name','first_name'] ,tablefmt='mysql'))

  store_id  last_name    first_name
----------  -----------  ------------
         1  SMITH        MARY
         1  JOHNSON      PATRICIA
         1  WILLIAMS     LINDA
         2  JONES        BARBARA
         1  BROWN        ELIZABETH


In [10]:
mycursor.execute("SELECT * FROM customer LIMIT 5 ")
myresult = mycursor.fetchall()
print(tabulate(myresult,tablefmt='mysql'))

-  -  ---------  --------  -----------------------------------  -  -  -------------------  -------------------
1  1  MARY       SMITH     MARY.SMITH@sakilacustomer.org        5  1  2006-02-14 22:04:36  2006-02-15 04:57:20
2  1  PATRICIA   JOHNSON   PATRICIA.JOHNSON@sakilacustomer.org  6  1  2006-02-14 22:04:36  2006-02-15 04:57:20
3  1  LINDA      WILLIAMS  LINDA.WILLIAMS@sakilacustomer.org    7  1  2006-02-14 22:04:36  2006-02-15 04:57:20
4  2  BARBARA    JONES     BARBARA.JONES@sakilacustomer.org     8  1  2006-02-14 22:04:36  2006-02-15 04:57:20
5  1  ELIZABETH  BROWN     ELIZABETH.BROWN@sakilacustomer.org   9  1  2006-02-14 22:04:36  2006-02-15 04:57:20
-  -  ---------  --------  -----------------------------------  -  -  -------------------  -------------------
